In [358]:
import os 
from typing import List, Tuple
import pickle
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
from datetime import datetime
import torch
import torch.nn as nn


In [359]:
class DINModel(nn.Module):
    def __init__(self, uid_num, gender_num, age_num, job_num, mid_num, movie_category_num, movie_title_num, max_hist_len, embedding_dim=16, attention_units=32):
        """初始化DIN模型"""
        super(DINModel, self).__init__()
        self.embedding_dim = embedding_dim
        
        # 用户特征嵌入层
        self.uid_embedding = nn.Embedding(uid_num, embedding_dim)
        self.gender_embedding = nn.Embedding(gender_num, embedding_dim//2)
        self.age_embedding = nn.Embedding(age_num, embedding_dim//2)
        self.job_embedding = nn.Embedding(job_num, embedding_dim//2)

        # 电影特征嵌入层
        self.movie_id_embedding = nn.Embedding(mid_num, embedding_dim)
        self.movie_categories_embedding = nn.Embedding(movie_category_num, embedding_dim)
        
        self.movie_title_embedding = nn.Embedding(movie_title_num, embedding_dim//2)

        # 历史电影特征嵌入层
        self.history_movie_embedding = nn.Embedding(max_hist_len, embedding_dim)
        # 注意力层
        self.attention = nn.Sequential(
            nn.Linear(embedding_dim * 2, attention_units),
            nn.ReLU(),
            nn.Linear(attention_units, attention_units),
            nn.ReLU(),
            nn.Linear(attention_units, 1),
            nn.Sigmoid()
        )
        
        # 预测层
        self.prediction = nn.Sequential(
            nn.Linear(57, 128),
            nn.ReLU(),
            nn.Linear(128, 64), 
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        # print("预测层维度:", self.prediction)
        
    def forward(self, uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles, history_movie_ids):
        """前向传播
        Args:
            uid: 用户ID [batch_size]
            user_gender: 用户性别 [batch_size]
            user_age: 用户年龄 [batch_size]
            user_job: 用户职业 [batch_size]
            movie_id: 候选电影ID [batch_size]
            movie_categories: 候选电影类别 [batch_size]
            movie_titles: 候选电影标题 [batch_size]
            history_movie_ids: 历史交互电影ID [batch_size, hist_len]
        """
        # 嵌入用户特征
        uid_embed = self.uid_embedding(uid) # torch.size([32, 16])
        # print("用户ID维度:", uid_embed.size())
        gender_embed = self.gender_embedding(user_gender) # torch.size([32, 8])
        # print("用户性别维度:", gender_embed.size())
        age_embed = self.age_embedding(user_age) # torch.size([32, 8])
        # print("年龄维度:", age_embed.size())
        job_embed = self.job_embedding(user_job) # torch.size([32, 8])
        # print("用户职业维度:", job_embed.size())
        
        # 嵌入候选电影特征
        movie_id_embed = self.movie_id_embedding(movie_id) # torch.size([32, 16])
        # print("电影ID维度:", movie_id_embed.size())
        movie_categories_embed = self.movie_categories_embedding(movie_categories) # torch.size([32, 18, 16])
        # print("电影类型维度:", movie_categories_embed.size())
        movie_titles_embed = self.movie_title_embedding(movie_titles) # torch.size([32, 15, 8])
        # print("电影标题维度:", movie_titles_embed.size())
        # print("电影标题的类型:", movie_titles_embed.dtype)
        # 嵌入历史电影ID特征
        # print("历史电影ID的类型:", history_movie_ids.dtype)
        hist_movie_embed = self.movie_id_embedding(history_movie_ids) # torch.size([32, 16])
        # print('历史电影ID维度:', hist_movie_embed.size())

        
        # 注意力机制处理历史交互
        attention_input = torch.cat([
            movie_id_embed,
            hist_movie_embed
        ], dim=-1) 

        attention_weight = self.attention(attention_input)
        hist_attention = torch.sum(attention_weight * hist_movie_embed, dim=1)
        
        # 拼接所有特征
        # print("uid_embed维度", uid_embed.size())
        # print("gender_embed维度", gender_embed.size())
        # print("age_embed维度:", age_embed.size())
        # print("job embed维度:", job_embed.size())
        # print("movie id embed维度:", movie_id_embed.size())
        # print("hist attention维度:", hist_attention.size())
        concat_features = torch.cat([
            uid_embed, gender_embed, age_embed, job_embed,
            movie_id_embed, hist_attention.unsqueeze(1)
        ], dim=1)
        # print("拼接后的特征维度:", concat_features.size())
        
        # 输出预测分数
        # print("输出数据的数据类型:", concat_features.dtype)
        return self.prediction(concat_features)


In [360]:
from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        self.max_hist_len = max(len(feature[8]) for feature in self.features) # 计算最大历史电影ID列表长度

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        uid = torch.tensor(self.features[idx][0])
        movie_id = torch.tensor(self.features[idx][1])
        user_gender = torch.tensor(self.features[idx][2])
        user_age = torch.tensor(self.features[idx][3])
        user_job = torch.tensor(self.features[idx][4])
        movie_titles = torch.tensor(self.features[idx][6])
        movie_categories = torch.tensor(self.features[idx][7])
        history_movie_ids = torch.tensor(self.features[idx][8])

        # 这里一定要对history_movie_ids进行填充操作，否则会出现报错（需要保证传入的长度都是一致的）
        # 填充操作
        padding_len = self.max_hist_len - len(history_movie_ids)
        if padding_len > 0:
            padding = torch.zeros(padding_len, dtype=torch.long)
            history_movie_ids = torch.cat((history_movie_ids, padding), dim=0)
        targets = torch.tensor(self.targets[idx]).float()
        return uid, movie_id, user_gender, user_age, user_job, movie_titles, movie_categories, history_movie_ids, targets

In [361]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from typing import List, Tuple
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备{device}")

class RankModel:
    def __init__(self, uid_num, gender_num, age_num, job_num, mid_num, movie_category_num, movie_title_num, max_hist_len):
        """初始化DIN排序模型"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 这里假设DINModel已经定义
        self.model = DINModel(uid_num, gender_num, age_num, job_num, mid_num, movie_category_num, movie_title_num, max_hist_len).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.criterion = nn.MSELoss()

    def train(self, train_loader, num_epochs=5):
        """训练模型
        Args:
            train_features: 训练集特征
            train_targets: 训练集标签
        """
        print("开始训练DIN排序模型...")
        for epoch in range(num_epochs):
            self.model.train()
            for batch_i, (uid, movie_id, user_gender, user_age, user_job, movie_titles, movie_categories, ratings, history_movie_ids) in enumerate(train_loader):
                self.optimizer.zero_grad()
                uid = uid.to(device)
                user_gender = user_gender.to(device)
                user_age = user_age.to(device)
                user_job = user_job.to(device)
                movie_id = movie_id.to(device)
                movie_categories = movie_categories.to(device)
                movie_titles = movie_titles.to(device)
                history_movie_ids = history_movie_ids.to(device).long() # 这里一定要设置long，否则会出现对history_movie_ids的类型报错，并且要扩充维度
                # print("ratings的数据类型:", ratings.dtype)
                ratings = ratings.float().to(device)

                # 前向传播
                outputs = self.model(uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles, history_movie_ids)
                loss = self.criterion(outputs, ratings)

                # 反向传播和优化
                loss.backward()
                self.optimizer.step()
                if (batch_i + 1) % 100 == 0:
                    print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        print("训练完成！")

    def evaluate(self, test_loader):
        """评估模型
        Args:
            test_loader: 测试集的DataLoader
        """
        self.model.eval()
        all_predictions = []
        all_ratings = []
        with torch.no_grad():
            for batch_i, (uid, movie_id, user_gender, user_age, user_job, movie_titles, movie_categories, ratings, history_movie_ids) in enumerate(test_loader):
                uid = uid.to(self.device)
                user_gender = user_gender.to(self.device)
                user_age = user_age.to(self.device)
                user_job = user_job.to(self.device)
                movie_id = movie_id.to(self.device)
                movie_categories = movie_categories.to(self.device)
                movie_titles = movie_titles.to(self.device)
                ratings = ratings.to(self.device)
                history_movie_ids=history_movie_ids.to(self.device)
                predictions = self.model(uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles)
                all_predictions.extend(predictions.cpu().numpy().flatten())
                all_ratings.extend(ratings.cpu().numpy().flatten())

        mse = mean_squared_error(all_ratings, all_predictions)
        mae = mean_absolute_error(all_ratings, all_predictions)
        print(f"测试集MSE: {mse:.4f}")
        print(f"测试集MAE: {mae:.4f}")

    def predict(self, features: np.ndarray) -> np.ndarray:
        """预测评分
        Args:
            features: 待预测特征
        Returns:
            预测的评分
        """
        self.model.eval()
        with torch.no_grad():
            uid = torch.tensor(features[:, 0].to(self.device))
            user_gender = torch.tensor(features[:, 2].to(self.device))
            user_age = torch.tensor(features[:, 3].to(self.device))
            user_job = torch.tensor(features[:, 4].to(self.device))
            movie_id = torch.tensor(features[:, 1].to(self.device))
            movie_categories = torch.tensor(features[:, 7].to(self.device))
            movie_titles = torch.tensor(features[:, 6].to(self.device))
            history_movie_ids = torch.tensor(features[:, 8].to(self.device))

            predictions = self.model(uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles, history_movie_ids)
            return predictions.cpu().numpy()

    def get_recommendations(self, user_features: np.ndarray, recall_movie_features: np.ndarray, top_k: int = 10) -> List[Tuple[int, float]]:
        """获取推荐电影列表
        Args:
            user_features: 用户特征
            recall_movie_features: 召回的候选电影特征
            top_k: 推荐电影数量
        Returns:
            推荐电影列表，每个元素为(电影ID, 预测评分)
        """
        predictions = self.predict(recall_movie_features)
        movie_scores = list(enumerate(predictions))
        movie_scores.sort(key=lambda x: x[1], reverse=True)
        return movie_scores[:top_k]


使用设备cpu


### 处理数据

In [362]:
train_features, train_targets, test_features, test_targets=pickle.load(open('./data/split_dataset.p', 'rb'))
title2int, title_count, title_set, genres2int, genres_map, features_pd, targets_pd, features, targets_values, ratings_df, users_df, movies_df, data = pickle.load(open('./data/preprocess.p', 'rb'))

In [363]:
data.head()

,user_id,movie_id,rating,gender,age,occupation,zip_code,title,genres
0,1,1193,5,1,0,10,48067,"[4835, 130, 3852, 2242, 1296, 4153, 3512, 3512...","[4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."
1,1,661,3,1,0,10,48067,"[4809, 4472, 2242, 3869, 1291, 3512, 3512, 351...","[11, 15, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7..."
2,1,914,3,1,0,10,48067,"[4394, 1320, 1007, 3512, 3512, 3512, 3512, 351...","[0, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."
3,1,3408,4,1,0,10,48067,"[1951, 4696, 3512, 3512, 3512, 3512, 3512, 351...","[4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."
4,1,2355,5,1,0,10,48067,"[722, 932, 973, 3512, 3512, 3512, 3512, 3512, ...","[11, 15, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7..."


In [364]:
user_movie_ids = data.groupby('user_id')['movie_id'].apply(list).reset_index()

In [365]:
user_movie_ids

,user_id,movie_id
0,1,"[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
1,2,"[1357, 3068, 1537, 647, 2194, 648, 2268, 2628,..."
2,3,"[3421, 1641, 648, 1394, 3534, 104, 2735, 1210,..."
3,4,"[3468, 1210, 2951, 1214, 1036, 260, 2028, 480,..."
4,5,"[2987, 2333, 1175, 39, 288, 2337, 1535, 1392, ..."
...,...,...
6035,6036,"[571, 574, 2053, 2054, 2058, 588, 589, 4, 3005..."
6036,6037,"[589, 3006, 1407, 2064, 2065, 593, 3015, 903, ..."
6037,6038,"[1419, 920, 3088, 232, 1136, 1148, 1183, 2146,..."
6038,6039,"[588, 2067, 1416, 3022, 3028, 2080, 2083, 2087..."


In [366]:
# 在data后增加一列，用来记录用户的历史观看的电影ID
new_data = pd.merge(data, user_movie_ids, on='user_id', how='left')

# 修改列名
new_data=new_data.rename(columns={'movie_id_y':'history_movie_ids', 'movie_id_x': 'movie_id'})


In [367]:
new_data.head()

,user_id,movie_id,rating,gender,age,occupation,zip_code,title,genres,history_movie_ids
0,1,1193,5,1,0,10,48067,"[4835, 130, 3852, 2242, 1296, 4153, 3512, 3512...","[4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
1,1,661,3,1,0,10,48067,"[4809, 4472, 2242, 3869, 1291, 3512, 3512, 351...","[11, 15, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7...","[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
2,1,914,3,1,0,10,48067,"[4394, 1320, 1007, 3512, 3512, 3512, 3512, 351...","[0, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
3,1,3408,4,1,0,10,48067,"[1951, 4696, 3512, 3512, 3512, 3512, 3512, 351...","[4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
4,1,2355,5,1,0,10,48067,"[722, 932, 973, 3512, 3512, 3512, 3512, 3512, ...","[11, 15, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7...","[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."


In [368]:
new_data.columns

Index(['user_id', 'movie_id', 'rating', 'gender', 'age', 'occupation',
       'zip_code', 'title', 'genres', 'history_movie_ids'],
      dtype='object')

### 训练模型

In [369]:
# 从data中划分训练集和测试集
print("开始划分训练集和测试集...")

# 将数据转换为numpy数组
features = np.array(new_data[['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip_code', 'title', 'genres', 'history_movie_ids']].values)
targets = np.array(new_data['rating'].values)

max_hist_len = max(len(feature[8]) for feature in features)
print("最长的历史电影ID数量为:", max_hist_len)


# 使用train_test_split划分训练集和测试集
from sklearn.model_selection import train_test_split
train_features, test_features, train_targets, test_targets = train_test_split(
    features, targets, test_size=0.2, random_state=42
)

print(f"训练集大小: {len(train_features)}")
print(f"测试集大小: {len(test_features)}")

train_dataset=MovieDataset(train_features, train_targets)
test_dataset=MovieDataset(test_features, test_targets)
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=32, shuffle=False)


开始划分训练集和测试集...
最长的历史电影ID数量为: 2314
训练集大小: 800167
测试集大小: 200042


In [370]:
embed_dim = 32
# 用户 ID 个数
uid_num = max(features.take(0, 1)) + 1
# 性别个数
gender_num = max(features.take(2, 1)) + 1
# 年龄类别个数
age_num = max(features.take(3, 1)) + 1
# 职业个数
job_num = max(features.take(4, 1)) + 1

# 电影 ID 个数
mid_num = max(features.take(1, 1)) + 1
# 电影类型个数
movie_category_num = max(genres2int.values()) + 1
# 电影名单词个数
movie_title_num = len(title_set)

# 初始化排序模型
rank_model = RankModel(uid_num, gender_num, age_num, job_num, mid_num, movie_category_num, movie_title_num, max_hist_len)

# 训练模型
print("开始训练模型...")
rank_model.train(train_loader)

# 评估模型
print("开始评估模型...")
rank_model.evaluate(test_features, test_targets)

# 保存模型
save_dir = os.path.join(os.getcwd(), "model_save")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取当前日期作为模型文件名的一部分
from datetime import datetime
today = datetime.today()
formatted_date = today.strftime('%Y%m%d')

预测层维度: Sequential(
  (0): Linear(in_features=57, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
)
开始训练模型...
开始训练DIN排序模型...


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 2314])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/5], Step [100/25006], Loss: 894242.8125
Epoch [1/5], Step [200/25006], Loss: 631314.6875
Epoch [1/5], Step [300/25006], Loss: 697570.6875
Epoch [1/5], Step [400/25006], Loss: 731132.3750
Epoch [1/5], Step [500/25006], Loss: 716335.6875
Epoch [1/5], Step [600/25006], Loss: 527838.3125
Epoch [1/5], Step [700/25006], Loss: 722588.3750
Epoch [1/5], Step [800/25006], Loss: 482378.8438
Epoch [1/5], Step [900/25006], Loss: 581806.5625
Epoch [1/5], Step [1000/25006], Loss: 643319.1250
Epoch [1/5], Step [1100/25006], Loss: 651358.2500
Epoch [1/5], Step [1200/25006], Loss: 629393.1250
Epoch [1/5], Step [1300/25006], Loss: 682834.6250
Epoch [1/5], Step [1400/25006], Loss: 681497.3125
Epoch [1/5], Step [1500/25006], Loss: 748875.6250
Epoch [1/5], Step [1600/25006], Loss: 648402.3125
Epoch [1/5], Step [1700/25006], Loss: 681988.5625
Epoch [1/5], Step [1800/25006], Loss: 590061.9375
Epoch [1/5], Step [1900/25006], Loss: 583934.3125
Epoch [1/5], Step [2000/25006], Loss: 664216.8750
Epoch [1/

KeyboardInterrupt: 

In [ ]:

# 评估模型
print("开始评估模型...")
rank_model.evaluate(test_features, test_targets)

# 保存模型
save_dir = os.path.join(os.getcwd(), "model_save")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 获取当前日期作为模型文件名的一部分
from datetime import datetime
today = datetime.today()
formatted_date = today.strftime('%Y%m%d')

# 保存模型
model_path = os.path.join(save_dir, f"rank_model_{formatted_date}.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(rank_model, f)
print(f'模型已保存到: {model_path}')

# 测试推荐功能
print("\n测试推荐功能:")
# 随机选择一个用户
test_user_idx = np.random.randint(0, len(test_features))
test_user_features = test_features[test_user_idx:test_user_idx+1]

# 首先使用召回模型获取候选集
with torch.no_grad():
    recall_candidates = recall_model.get_recall_candidates(test_user_features, top_k=100)

# 获取召回候选集的特征
recall_movie_features = test_features[recall_candidates]

# 使用排序模型对召回结果进行排序
recommendations = rank_model.get_recommendations(test_user_features, recall_movie_features, top_k=5)
print(f"为用户推荐的Top5电影ID及预测评分:")
for movie_id, score in recommendations:
    print(f"电影ID: {movie_id}, 预测评分: {score:.2f}")
